In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import category_encoders
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import datetime
import gc,os
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import pickle

from mlxtend.regressor import StackingRegressor
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

os.chdir('../input/')
weather_train=pd.read_csv('ashrae-energy-prediction/weather_train.csv')
weather_test=pd.read_csv('ashrae-energy-prediction/weather_test.csv')
train=pd.read_csv('ashrae-energy-prediction/train.csv')
test=pd.read_csv('ashrae-energy-prediction/test.csv')
building_metadata=pd.read_csv('ashrae-energy-prediction/building_metadata.csv')

#building_metadata,weather_test and weather_train have null values which have to be fixed.
#next look for outliers and remove them
train=train[train['building_id']!=1099]
train=train.query('not(building_id<=104 & meter==0 & timestamp<="2016-05-20")')

def fill_weather_dataset(weather_train)->pd.DataFrame:
    time_format = '%Y-%m-%d %H:%M:%S'
    start_date = datetime.datetime.strptime(weather_train['timestamp'].min(), time_format)
    end_date = datetime.datetime.strptime(weather_train['timestamp'].max(), time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    for site_id in range(16):
        site_hours = np.array(weather_train[weather_train['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list, site_hours), columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_train = pd.concat([weather_train, new_rows], sort=True)
        weather_train = weather_train.reset_index(drop=True)  
    # Add new Features
    weather_train["datetime"] = pd.to_datetime(weather_train["timestamp"])
    weather_train["day"] = weather_train["datetime"].dt.day
    weather_train["week"] = weather_train["datetime"].dt.week
    weather_train["month"] = weather_train["datetime"].dt.month
     # Reset Index for Fast Update
    weather_train = weather_train.set_index(['site_id','day','month'])
    air_temperature_filler = pd.DataFrame(weather_train.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_train.update(air_temperature_filler,overwrite=False)
    # Step 1
    cloud_coverage_filler = weather_train.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_train.update(cloud_coverage_filler,overwrite=False)
    dew_temperature_filler = pd.DataFrame(weather_train.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_train.update(dew_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_train.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_train.update(sea_level_filler,overwrite=False)
    wind_direction_filler =  pd.DataFrame(weather_train.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_train.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_train.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_train.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_train.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_train.update(precip_depth_filler,overwrite=False)

    weather_train = weather_train.reset_index()
    weather_train = weather_train.drop(['datetime','day','week','month'],axis=1)
        
    return weather_train

def reduce_mem_usage(df, use_float16=False)->pd.DataFrame:
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def features_engineering(df)->pd.DataFrame:
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                    "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                    "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                    "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                    "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                    "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                    "2019-01-01"]
    df["is_holiday"] = (df.timestamp.isin(holidays)).astype(int)
    df['square_feet'] =  np.log1p(df['square_feet'])
    
    # Remove Unused Columns
    drop = ["timestamp","sea_level_pressure", "wind_direction", "wind_speed","year_built","floor_count"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

weather_train = fill_weather_dataset(weather_train)
train= reduce_mem_usage(train,use_float16=True)
building_metadata = reduce_mem_usage(building_metadata,use_float16=True)
weather_train = reduce_mem_usage(weather_train,use_float16=True)
train = train.merge(building_metadata,on='building_id',how='left')
train = train.merge(weather_train,how='left',on=['site_id','timestamp'])
del weather_train
gc.collect()
train = features_engineering(train)

target = np.log1p(train["meter_reading"])
train_df = train.drop(['meter_reading'], axis = 1)
del train
gc.collect()
train_df.head()

categorical_features = ["building_id", "site_id", "meter", "primary_use"]
ce = category_encoders.CountEncoder(cols=categorical_features)
ce.fit(train_df)
train_df = ce.transform(train_df)
N_train = train_df.shape[0]
for feature in categorical_features:
    train_df[feature] = train_df[feature]/N_train
    
# Missing data imputation
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(train_df)
train_df = imputer.transform(train_df)

# Regressors
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
kf = KFold(n_splits=2,shuffle=False)
models = []
for train_index, val_index in kf.split(train_df):
    train_features = train_df[train_index]
    train_target = target[train_index]    
    val_features = train_df[val_index]
    val_target = target[val_index]
    model = LinearRegression()    
    model.fit(np.array(train_features),np.array(train_target))
    models.append(model)        
    val_pred = model.predict(val_features)
    print(np.sqrt(mean_squared_error(val_target, val_pred)))
    del train_features, train_target, val_features, val_target

del train_df, target    
gc.collect()

lightgbm = LGBMRegressor(objective='regression', learning_rate=0.05, num_leaves=1024,
    feature_fraction=0.8, bagging_fraction=0.9, bagging_freq=5) 

ridge = Ridge(alpha=0.3)
lasso = Lasso(alpha=0.3)

kf = KFold(n_splits=2,shuffle=False)
models = []
for train_index, val_index in kf.split(train_df):
    train_features = train_df[train_index]
    train_target = target[train_index]
    
    val_features = train_df[val_index]
    val_target = target[val_index]
    
    model = StackingRegressor(regressors=(lightgbm, ridge, lasso),
        meta_regressor=lightgbm, use_features_in_secondary=True)    
    model.fit(np.array(train_features),np.array(train_target))
    models.append(model)
    val_pred = model.predict(val_features)
    print(np.sqrt(mean_squared_error(val_target, val_pred)))
    del train_features, train_target, val_features, val_target

del train_df, target    
gc.collect()
#test
row_ids = test['row_id']
test.drop('row_id', axis=1, inplace=True)
weather_test = fill_weather_dataset(weather_test)
# Memory optimization
test = reduce_mem_usage(test, use_float16=True)
weather_test = reduce_mem_usage(weather_test, use_float16=True)
# Merge test data
test = test.merge(building_metadata, on='building_id', how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

del building_metadata
del weather_test
gc.collect()

# Test data processing
test = features_engineering(test)

test = ce.transform(test)
for feature in categorical_features:
    test[feature] = test[feature] / N_train

test = imputer.transform(test)

# Make predictions
predictions = 0
for model in models:
    predictions += np.expm1(model.predict(np.array(test))) / len(models)
    del model; gc.collect()

del test, models; gc.collect()



# Create submission file
submission = pd.DataFrame({
    'row_id': row_ids,
    'meter_reading': np.clip(predictions, 0, a_max=None)
})
submission.to_csv('/kaggle/working/submission.csv', index=False, float_format='%.4f')'''

